In [1]:
import lsst.daf.persistence as dafPersist
import matplotlib.pyplot as plt
import numpy as np

import lsst.afw.table as afwTable
import lsst.afw.geom as afwGeom
import Utils.DiaSourceTools as DSTools

In [4]:
from astropy.table import Table

In [10]:
t = Table()
t = Table(names=('a', 'b', 'c'), dtype=('S2', 'S2', 'S2'))
t.add_row(('y', 'z', 'x'))

In [11]:
print t

 a   b   c 
--- --- ---
  y   z   x


In [12]:
DATADIR="/renoir_data_02/jpreyes/lsst_data/CFHTLS_sim/"

In [13]:
import pickle

out = open("/renoir_data_02/jpreyes/lsst_data/calexp_coords.pickle", "rb")
coords = pickle.load(out)

In [14]:
ccd = 14
parameters_order = range(0,9)
directory = DATADIR+"deepDiffTests20/SpatialKernelOrders/deepDiffTest300/"
butler_test20_cellsize300 = dafPersist.Butler(directory) 
visit = 850586


In [87]:
parameters_order = range(0,9,1)
print parameters_order

[0, 1, 2, 3, 4, 5, 6, 7, 8]


In [88]:
butlers20_cellSize = []


for par in parameters_order:
    directory = DATADIR+"deepDiffTests20/SpatialKernelOrders/deepDiffTest"+str(par)+"/"
    butlers20_cellSize.append(dafPersist.Butler(directory))


In [89]:
butlers15_cellSize = []



for par in parameters_order:
    directory = DATADIR+"deepDiffTests15/SpatialKernelOrders/deepDiffTest"+str(par)+"/"
    butlers15_cellSize.append(dafPersist.Butler(directory))

In [90]:
butlers10_cellSize = []



for par in parameters_order:
    directory = DATADIR+"deepDiffTests10/SpatialKernelOrders/deepDiffTest"+str(par)+"/"
    butlers10_cellSize.append(dafPersist.Butler(directory))


In [91]:
butlers5_cellSize = []


for par in parameters_order:
    directory = DATADIR+"deepDiffTests5/SpatialKernelOrders/deepDiffTest"+str(par)+"/"
    butlers5_cellSize.append(dafPersist.Butler(directory))


In [92]:
butlers3_cellSize = []



for par in parameters_order:
    directory = DATADIR+"deepDiffTests3/SpatialKernelOrders/deepDiffTest"+str(par)+"/"
    butlers3_cellSize.append(dafPersist.Butler(directory))


In [21]:
ccd = 14
visit = 850586
dataId = {"ccd":ccd,"filter":'r', "visit":visit}


In [22]:
def get_detected_sn(butler, dataId, coords):
    
    class_diaSrc = butler.get("deepDiff_class_diaSrc", dataId)
    diffexp = butler.get("deepDiff_differenceExp", dataId)
    wcs = diffexp.getWcs()
    count = 0
    detected=[]
    for source in class_diaSrc:
        for x, y in coords:
            radec = wcs.pixelToSky(x,y)
            s1 = {"ra": source["coord_ra"], "dec": source["coord_dec"]}
            s2 = {"ra": radec.getLongitude(), "dec": radec.getLatitude()}
            if DSTools.source_distance(s1, s2) < 1.0:
                count+=1
                detected.append( wcs.skyToPixel(source["coord_ra"], source["coord_dec"]) )
                break

    return count
    

In [29]:
def get_sn_tags(butler, dataId, coords):
    
    class_diaSrc = butler.get("deepDiff_class_diaSrc", dataId)
    diffexp = butler.get("deepDiff_differenceExp", dataId)
    wcs = diffexp.getWcs()
    count = 0
    tags=[]
    for source in class_diaSrc:
        for x, y in coords:
            radec = wcs.pixelToSky(x,y)
            s1 = {"ra": source["coord_ra"], "dec": source["coord_dec"]}
            s2 = {"ra": radec.getLongitude(), "dec": radec.getLatitude()}
            if DSTools.source_distance(s1, s2) < 1.0:
                count+=1
                tags.append( source["classification_dipole"] )
                break

    return tags

def get_tags_count(tags):
    flags_dict = {}
    for i in range (0,8):
        flags_dict[i] = 0
        
    for tag in tags:
            flags_dict[tag] +=1
            
    return flags_dict

In [36]:
t = Table(names = ("a", "b", "c"))
t.add_row([1,2,3])
print t


 a   b   c 
--- --- ---
1.0 2.0 3.0


In [93]:
t20cs = Table()
t20cs = Table(names=('parameter', '0', '1', '2', '3', '4', '5', '6', '7', 'total_diaSrc'), dtype=('i4', 'i4', 'i4', 'i4', 'i4', 'i4', 'i4' , 'i4', 'i4', 'i4'))


In [94]:
for par, butler in zip(parameters_order, butlers20_cellSize):
    class_diaSrc = butler.get("deepDiff_class_diaSrc", dataId)
    tags = get_sn_tags(butler, dataId, coords)
    c_tags = get_tags_count(tags)
    def_arr = [par] + c_tags.values() + [len(class_diaSrc)]
    
    t20cs.add_row(def_arr)
    print def_arr

[0, 0, 0, 0, 5, 0, 0, 943, 12, 1256]
[1, 3, 0, 0, 2, 0, 0, 942, 10, 1282]
[2, 1, 1, 0, 2, 0, 0, 955, 10, 1122]
[3, 0, 0, 0, 1, 0, 0, 0, 0, 329]
[4, 0, 0, 0, 1, 0, 0, 0, 0, 329]
[5, 0, 0, 0, 1, 0, 0, 0, 0, 329]
[6, 0, 0, 0, 1, 0, 0, 0, 0, 329]
[7, 0, 0, 0, 1, 0, 0, 0, 0, 329]
[8, 0, 0, 0, 1, 0, 0, 0, 0, 329]


In [95]:
t15cs = Table()
t15cs = Table(names=('parameter', '0', '1', '2', '3', '4', '5', '6', '7', 'total_diaSrc'), dtype=('i4', 'i4', 'i4', 'i4', 'i4', 'i4', 'i4' , 'i4', 'i4', 'i4'))


In [96]:
for par, butler in zip(parameters_order, butlers15_cellSize):
    class_diaSrc = butler.get("deepDiff_class_diaSrc", dataId)
    tags = get_sn_tags(butler, dataId, coords)
    c_tags = get_tags_count(tags)
    def_arr = [par] + c_tags.values() + [len(class_diaSrc)]
    
    t15cs.add_row(def_arr)
    print def_arr

[0, 0, 0, 0, 6, 0, 0, 947, 10, 1258]
[1, 3, 0, 0, 4, 0, 0, 942, 10, 1283]
[2, 1, 0, 0, 1, 0, 0, 936, 11, 1104]
[3, 0, 0, 0, 1, 0, 0, 0, 0, 329]
[4, 0, 0, 0, 1, 0, 0, 0, 0, 329]
[5, 0, 0, 0, 1, 0, 0, 0, 0, 329]
[6, 0, 0, 0, 1, 0, 0, 0, 0, 329]
[7, 0, 0, 0, 1, 0, 0, 0, 0, 329]
[8, 0, 0, 0, 1, 0, 0, 0, 0, 329]


In [97]:
t10cs = Table()
t10cs = Table(names=('parameter', '0', '1', '2', '3', '4', '5', '6', '7', 'total_diaSrc'), dtype=('i4', 'i4', 'i4', 'i4', 'i4', 'i4', 'i4' , 'i4', 'i4', 'i4'))


In [98]:
for par, butler in zip(parameters_order, butlers10_cellSize):
    class_diaSrc = butler.get("deepDiff_class_diaSrc", dataId)
    tags = get_sn_tags(butler, dataId, coords)
    c_tags = get_tags_count(tags)
    def_arr = [par] + c_tags.values() + [len(class_diaSrc)]
    
    t10cs.add_row(def_arr)
    print def_arr

[0, 0, 0, 0, 4, 0, 0, 950, 11, 1259]
[1, 3, 0, 0, 3, 0, 0, 945, 10, 1284]
[2, 1, 0, 0, 2, 0, 0, 838, 17, 1013]
[3, 0, 0, 0, 1, 0, 0, 0, 0, 329]
[4, 0, 0, 0, 1, 0, 0, 0, 0, 329]
[5, 0, 0, 0, 1, 0, 0, 0, 0, 329]
[6, 0, 0, 0, 1, 0, 0, 0, 0, 329]
[7, 0, 0, 0, 1, 0, 0, 0, 0, 329]
[8, 0, 0, 0, 1, 0, 0, 0, 0, 329]


In [99]:
t5cs = Table()
t5cs = Table(names=('parameter', '0', '1', '2', '3', '4', '5', '6', '7', 'total_diaSrc'), dtype=('i4', 'i4', 'i4', 'i4', 'i4', 'i4', 'i4' , 'i4', 'i4', 'i4'))


In [100]:
for par, butler in zip(parameters_order, butlers5_cellSize):
    class_diaSrc = butler.get("deepDiff_class_diaSrc", dataId)
    tags = get_sn_tags(butler, dataId, coords)
    c_tags = get_tags_count(tags)
    def_arr = [par] + c_tags.values() + [len(class_diaSrc)]
    
    t5cs.add_row(def_arr)
    print def_arr

[0, 2, 0, 0, 0, 0, 0, 698, 87, 1084]
[1, 4, 0, 0, 2, 0, 0, 653, 69, 1052]
[2, 1, 0, 0, 0, 0, 0, 366, 45, 569]
[3, 0, 0, 0, 1, 0, 0, 0, 0, 329]
[4, 0, 0, 0, 1, 0, 0, 0, 0, 329]
[5, 0, 0, 0, 1, 0, 0, 0, 0, 329]
[6, 0, 0, 0, 1, 0, 0, 0, 0, 329]
[7, 0, 0, 0, 1, 0, 0, 0, 0, 329]
[8, 0, 0, 0, 1, 0, 0, 0, 0, 329]


In [101]:
t3cs = Table()
t3cs = Table(names=('parameter', '0', '1', '2', '3', '4', '5', '6', '7', 'total_diaSrc'), dtype=('i4', 'i4', 'i4', 'i4', 'i4', 'i4', 'i4' , 'i4', 'i4', 'i4'))


In [ ]:
for par, butler in zip(parameters_order, butlers3_cellSize):
    class_diaSrc = butler.get("deepDiff_class_diaSrc", dataId)
    tags = get_sn_tags(butler, dataId, coords)
    c_tags = get_tags_count(tags)
    def_arr = [par] + c_tags.values() + [len(class_diaSrc)]
    
    t3cs.add_row(def_arr)
    print def_arr

[0, 0, 0, 0, 0, 0, 0, 33, 71, 405]
[1, 0, 0, 0, 0, 0, 0, 30, 63, 425]
[2, 0, 1, 0, 0, 0, 0, 9, 18, 187]
[3, 0, 0, 0, 1, 0, 0, 0, 0, 329]
[4, 0, 0, 0, 1, 0, 0, 0, 0, 329]


In [ ]:
t20cs.write("t20sko.fits")
t15cs.write("t15sko.fits")
t10cs.write("t10sko.fits")
t5cs.write("t5sko.fits")
t3cs.write("t3sko.fits")

In [31]:
len(butlers3_cellSize)

19

In [ ]:
butlers20_cellSize = []

parameters_order = range(50,525, 25)

for par in parameters_order:
    directory = DATADIR+"deepDiffTests20/CellSizes/deepDiffTest"+str(par)+"/"
    butlers20_cellSize.append(dafPersist.Butler(directory))

In [41]:
 class_diaSrc = butler.get("deepDiff_class_diaSrc", dataId)

In [42]:
class_diaSrc[0].schema

Schema(
    (Field['L'](name="id", doc="unique ID"), Key<L>(offset=0, nElements=1)),
    (Field['Angle'](name="coord_ra", doc="position in ra/dec"), Key<Angle>(offset=8, nElements=1)),
    (Field['Angle'](name="coord_dec", doc="position in ra/dec"), Key<Angle>(offset=16, nElements=1)),
    (Field['L'](name="parent", doc="unique ID of parent source"), Key<L>(offset=24, nElements=1)),
    (Field['F'](name="classification_dipole", doc="probability of being a dipole"), Key<F>(offset=32, nElements=1)),
    (Field['Flag'](name="ip_diffim_NaiveDipoleCentroid_flag", doc="general failure flag, set if anything went wrong"), Key['Flag'](offset=40, bit=0)),
    (Field['Flag'](name="ip_diffim_NaiveDipoleCentroid_pos_flag", doc="failure flag for positive, set if anything went wrong"), Key['Flag'](offset=40, bit=1)),
    (Field['Flag'](name="ip_diffim_NaiveDipoleCentroid_neg_flag", doc="failure flag for negative, set if anything went wrong"), Key['Flag'](offset=40, bit=2)),
    (Field['D'](name="ip_d